<a href="https://colab.research.google.com/github/nikhilmishra1710/DIP/blob/main/histogram_equalisation/histogram_equalisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [15]:
def load_image(path):
  img = cv2.imread(path)
  image_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  initial_matrix_rgb=np.array(image_rgb)

  plt.imshow(initial_matrix_rgb)
  plt.show()

  dimensions = img.shape
  print(dimensions)

  print('Height of Image:', dimensions[0])
  print('Width of Image:', dimensions[1])
  print('Number of Channels:', dimensions[2])

  return initial_matrix_rgb


In [16]:
def convert_to_grayscale(initial_matrix_rgb):
  row = initial_matrix_rgb.shape[0]
  col = initial_matrix_rgb.shape[1]
  print(row,col)
  initial_matrix_gray = np.zeros((row,col),dtype="int")

  for i in range(row):
    for j in range(col):
      initial_matrix_gray[i,j]=int(round((initial_matrix_rgb[i,j,0]+initial_matrix_rgb[i,j,1]+initial_matrix_rgb[i,j,2])/3))

  plt.imshow(initial_matrix_gray,cmap="gray")
  plt.show()

  return initial_matrix_gray

In [17]:
def calculate_cdf(initial_matrix_gray):
    values = np.unique(initial_matrix_gray)
    print("Unique pixel values:", values)

    frequency = {}
    for value in values:
        frequency[value] = np.count_nonzero(initial_matrix_gray == value)

    print("Frequency of pixel values:", frequency)

    cdf = {}
    cdf[values[0]] = frequency[values[0]]

    for i in range(1, len(values)):
        cdf[values[i]] = cdf[values[i - 1]] + frequency[values[i]]

    print("Cumulative Distribution Function (CDF):", cdf)
    display_histogram(values,frequency.values(),'Original Image Histogram')
    display_cdf_curve(values,cdf.values(),'Original Image CDF')
    return cdf, values, frequency

In [18]:
def display_histogram(values,frequency,title):
  plt.bar(values, frequency, width=1, color='gray', edgecolor='black',label="Pixel frequency")
  plt.title(title)
  plt.xlabel('Pixel Value')
  plt.ylabel('Frequency')
  plt.xlim([0, 255])
  plt.xticks(np.arange(0, 256, 16))
  plt.grid(axis='y')
  plt.show()

def display_cdf_curve(values,cumulative_frequency,title):
  plt.plot(values, cumulative_frequency, color='blue', label='CDF')
  plt.title(title)
  plt.xlabel('Pixel Value')
  plt.ylabel('Cumulative Frequency')
  plt.xlim([0, 255])
  plt.xticks(np.arange(0, 256, 16))
  plt.grid(axis='y')
  plt.show()

In [19]:
def calculate_h(v,cdf,m,n,L=256):
    min_cdf = min(cdf.values())
    return round((L-1)*(cdf[v]-min_cdf)/((m*n)-min_cdf))

In [20]:
def equalize_matrix(initial_matrix_gray,cdf,values,frequency):
  h={}
  row = initial_matrix_gray.shape[0]
  col = initial_matrix_gray.shape[1]
  for value in values:
      h[value]=calculate_h(value,cdf,row,col)

  equalized_matrix_gray = np.zeros_like(initial_matrix_gray)

  for key, value in h.items():
      equalized_matrix_gray[initial_matrix_gray == key] = value

  print("Equalisied Cumulative Distribution Function (CDF):", h)
  display_histogram(h.values(),frequency.values(),'Equalized Image Histogram')
  display_cdf_curve(h.values(),h.values(),'Equalized Image CDF')
  plt.imshow(equalized_matrix_gray,cmap="gray")
  plt.show()

  return equalized_matrix_gray

In [21]:
def histogram_equalization(path,new_file_name):
  initial_matrix_rgb=load_image(path)
  initial_matrix_gray=convert_to_grayscale(initial_matrix_rgb)
  cdf,values,frequency=calculate_cdf(initial_matrix_gray)
  equalized_img = equalize_matrix(initial_matrix_gray,cdf,values,frequency)
  cv2.imwrite(new_file_name, equalized_img)
  return equalized_img



In [ ]:
initial_image_matrix = np.array([[52,	55,	61,	59,	79,	61,	76,	61],
[62,	59,	55,	104, 94	,85	,59	,71],
[63,	65,	66,	113, 144,104,	63,	72],
[64,	70,	70,	126, 154,109,	71,	69],
[67,	73,	68,	106, 122,88	,68	,68],
[68,	79,	60,	70, 77	,66	,58	,75],
[69,	85,	64,	58, 55	,61	,65	,83],
[70,	87,	69,	68, 65	,73	,78	,90]
])
plt.imshow(initial_image_matrix,cmap="gray")
plt.show()
cv2.imwrite("/content/image.png", initial_image_matrix)
cdf,values,frequency=calculate_cdf(initial_image_matrix)
equalized_img = equalize_matrix(initial_image_matrix,cdf,values,frequency)
cv2.imwrite("/content/equalized_image4.png", equalized_img)


In [ ]:
final_img=histogram_equalization('/content/image.png','/content/equalized_image.png')

In [ ]:
final_img = histogram_equalization('/content/lena.png','/content/equalized_image_lena.png')

In [ ]:
final_img = histogram_equalization('/content/image3.jpg','/content/equalized_image3.png')